In [ ]:
%run ../input/arc-notebooks/5-crop-tasks-by-brute-force.ipynb
!mv submission.csv crop-sub.csv

%run ../input/arc-notebooks/genetic-dsl-part-from-10th-place-python-version.ipynb
!mv submission.csv genetic-sub.csv

%run ../input/arc-notebooks/repairingmosaicsandsymmetry.ipynb
!mv submission.csv mosai-sub.csv

%run ../input/endtoendthirdplacesolution/endtoend.ipynb
!mv submission-0.csv sub-0.csv
!mv submission-1.csv sub-1.csv
!mv submission-2.csv sub-2.csv
!mv submission-3.csv sub-3.csv
!mv submission-4.csv sub-4.csv
!mv submission-5.csv sub-5.csv

%run ../input/arc-notebooks/arc_1st_place_solution.py
!mv submission.csv icecuber-sub.csv

In [ ]:
import pandas as pd
import math
import random
import numpy as np
from random import seed

In [ ]:
seed(5)

In [ ]:
ss_path = '/kaggle/input/abstraction-and-reasoning-challenge/sample_submission.csv'
idx = pd.read_csv(ss_path).output_id.values

In [ ]:
def remove_within_dups_from_row(row):
    preds = row.split(' ')
    return ' '.join([
        pred for i, pred in enumerate(preds) if preds.index(pred) == i
    ])

def remove_within_dups(sub):
    sub_new = sub.copy()
    sub_new.output = sub.output.apply(remove_within_dups_from_row)
    return sub_new

In [ ]:
def get_sub(nick):
    try:
        sub = pd.read_csv(f'./{nick}.csv')
        if 'output_id' in sub.columns:
            sub = sub.set_index('output_id')
        sub.index.name = 'output_id'
        if not isinstance(sub.index[0], str):
            sub.index = pd.read_csv(ss_path).output_id.values
        sub = sub.fillna('')
        for c in ['|0|', '|0| |0| |0|', '|123|456|789|']:
            sub = sub.replace(c, '')
        sub['output'] = sub['output'].str.strip()
        sub = remove_within_dups(sub)
    except:
        sub = pd.DataFrame(columns=['output'])
        sub.index.name = 'output_id'
    return sub

In [ ]:
def combine(subs, first_only, order_by_confidence):
    if order_by_confidence:
        f = lambda df: df[df['output'] != ''].shape[0]
        subs = sorted(subs, key=f)
    submission = pd.DataFrame(columns=['output'])
    submission.index.name = 'output_id'
    for i, df in enumerate(subs):
        output_ids = set(df[df['output'] != ''].index)
        for output_id in output_ids:
            current = df.loc[output_id, 'output']
            if first_only and i != len(subs) - 1:
                current = current.split(' ')[0]
            if output_id in submission.index:
                submission.loc[output_id, 'output'] = submission.loc[output_id, 'output'] + ' ' + current
            else:
                submission.loc[output_id, 'output'] = current 
    return submission

In [ ]:
def top_three(preds_str):
    preds, counts = [], []
    for pred in preds_str.split(' '):
        if pred in preds:
            counts[preds.index(pred)] += 1
        else:
            preds.append(pred)
            counts.append(1)
    preds_capped = []
    p, c = [p for p in preds], [c for c in counts]
    while len(preds_capped) < 3 and len(p) > 0:
        mxcount = max(c)
        idx = c.index(mxcount)
        preds_capped.append(p[idx])
        p = [x for i, x in enumerate(p) if i != idx]
        c = [x for i, x in enumerate(c) if i != idx]
    return ' '.join(preds_capped)

In [ ]:
def main(nicks, first_only, majority_voting, order_by_confidence):
    subs = [get_sub(nick) for nick in nicks]
    sub = combine(subs, first_only=first_only, order_by_confidence=order_by_confidence)
    submission = sub.copy()
    if majority_voting:
        submission.output = submission.output.apply(top_three)
    else:
        submission = remove_within_dups(submission)
        f = lambda s: ' '.join(s.split(' ')[:3])
        submission.output = submission.output.apply(f)
    submission.to_csv('submission.csv')
    return submission

In [ ]:
nicks = [
    'crop-sub',
    'genetic-sub',
    'mosai-sub',
    'sub-0',
    'sub-1',
    'sub-2',
    'sub-3',
    'sub-4',
    'sub-5',
    'icecuber-sub'
]

submission = main(
    nicks=nicks,
    first_only=True,
    majority_voting=False,
    order_by_confidence=True
)